In [167]:
import pandas as pd
import flair
from sklearn.pipeline import Pipeline
from bpemb import BPEmb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
from nltk.stem import WordNetLemmatizer
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN
from scipy.spatial.distance import cdist
from sklearn.preprocessing import OneHotEncoder

In [30]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/londogard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/londogard/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [49]:
keywords = {'wireshark', 'cyber', 'docker', 'kanban', 'latest technologies', 'teaching skills', 'jenkins', 'sae j2847', 'system design', 'android', 'ccc', 'measurements', 'thermal conditioning', 'product development process', 'sales', 'verification tests', 'plc', 'test automation', 'private equity', 'data handling', 'maven', 'in-vehicle software', 'project management', 'validate', 'autonomous driver', 'insurances', 'drug', 'problem solver', 'infotainment', 'sensor fusion', 'ats', 'documentation', 'integration transformation', 'test planning', 'wallet-sizing analysis', 'software requirement', 'propulsion', 'test plans', 'canalyzer', 'nvidia', 'research and development', 'team efficiency', 'modems', 'automated test', 'matlab simulink', 'spice', 'lead', 'canoe', 'stk', 'control', 'product owner', 'lin', 'signal processing', 'insurance', 'frontend', 'customer', 'mil', 'backend', 'github', 'database', 'engineering tests', 'web services', 'automotive standards', 'valuation gap', 'university degree', 'software', 'cabel harness', 'application maintenance', 'sw test', 'collaborative', 'engineering', 'pcb and manufacturing', 'documenting processes', 'planning', 'safety requirement', 'coordinate', 'microprocessor system', 'interfaces', 'javascript', 'communication skills', 'user intent algorithms', 'target setting', 'rtos', 'hil', 'devops', 'digital design', 'manufacturing', 'electrical engineering', 'fund', 'sensors', 'soc', 'front-end', '5+', 'kola', 'network', 'gerrit', 'financial', 'signals', 'recruitment', 'low-level', 'phd', '>5', 'ci/cd', 'mcu', 'design for immunity', 'bus chassis', 'integration testing', 'ad/adas', 'stakeholders', 'm.sc.', 'equity analyst', 'bts', 'gpu development', 'back-end', 'artifactory', 'verbal', 'sae j3068', 'biopharmaceutical', 'offsite', 'schematic design', 'coverage model', 'etas inca', 'standardized requirement work', 'can', 'networking', 'data-centric', 'stake holder communication', 'erp system', 'electronics', 'saber', 'logistics systems', 'telecom', 'mechatronics', 'adas', 'polyspace', 'controller', 'quality assurance', 'coaching', 'canalayzer', 'c', 'invoicing', 'machine learning', 'train management', 'system lead', 'bachelor', 'technical', 'manage', 'windows', 'cmmi', 'project manager', 'cross-functional', 'management consulting', 'warranty', 'powertrain', 'end-to-end', 'bash', 'iso 15118', 'drivetrain', 'test programs', 'systems development', 'coordinating', 'robot framework', 'hazard analysis', 'oas', 'system testing', 'sensor perception layer', 'functional safety manager', 'report', 'efficiency', 'html', 'emc certification', 'uwb', 'it', 'diagnostics', 'esd', 'designer', 'oracle', 'testweaver', 'developer', 'healthcare', 'portfolio strategy', 'technical debt', 'confluence', 'unit testing', 'architecture', 'client discussion', '7740', 'sts', 'supply chain designer', 'electric motors', 'global coordination', 'aerospace', 'academic degree', 'e2e', 'efficient', 'flexray', 'financial modelling', 'embedded software', 'certification tests', 'design', 'business experience', 'configuration management', 'php', 'mechanical engineering', 'elektra', 'cloud', 'computer engineering', 'retail', 'vehicle electronics', 'eeprom', 'drug substance', 'high speed signals', 'power company', 'industrialization', 'filter designs', 'test', 'sap', 'css', 'system control', 'implementation', 'inverters', '5-8 years', 'validation', 'ad', 'electronics designer', 'application development', 'activism defense', 'front end', 'back end', 'sil1', 'cable harness engineer', 'it solutions', 'apriso', 'r&d', 'sql', 'wso2', 'software architect', 'track record', 'cd', 'business development', 'security', 'phone as key', 'jira', 'banking', 'system simulators', 'embedded system', 'hedging', 'sil', 'embedded systems', "master's degree", 'r3', 'pl', 'autonomous', 'app-centric', 'team lead', 'autosar', 'ai', 'bidding', 'traction', 'safe', 'budget', 'databases', 'international', 'rest api', 'saas', 'swedish', 'order management', 'automation', 'cts', 'budgeting', 'cameras', 'ccs', 'git', 'embedded', 'analog design', 'sw', 'emc', 'quality', 'customer value', 'vehicle certification', 'e-mobility', 'team leader', 'system verification', 'product manager', 'batteries', 'driving', '> 5', 'international business', 'scm', 'financial institutions', 'advanced driver assistance system', 'carve-out', 'team work', 'b.sc.', 'battery', 'dc/dc', 'linux', 'functional development', 'research & development', 'iso26262', 'home office', 'vehicle', 'operational team management', 'workshops', 'warranties', 'optimization', 'debugging', 'carweaver', 'hardware', 'pcb-a', 'italian', 'system development', 'din 70121', 'claim management', 'software developer', 'review', 'business', 'logistics', 'asil', 'computer science', 'cyber security', 'verification', 'chargers', 'vts', 'initiative', 'keycloak', 'qnx', 'carcom', 'ultrasonic perception', 'strategic vulnerabilities', 'payments', 'design for emissions', 'c/c++', 'bid teams', 'epsm', 'simulink', 'hil testing', 'angular', 'python', 'restful', 'sensor signaling', 'financing', 'digital key', 'fintech', 'operational', 'user experience', 'leader', 'railway', 'pcb', 'polarion', 'portuguese', 'continuous integration', 'radars', 'design guidelines', 'ethernet', 'electronic designer', 'vehicle dynamics', 'mba', 'matlab', 'economy', 'testing methodology', 'capl', 'java', 'master of science', 'ram', '10+', 'automotive', 'traction batteries', 'management', 'driving license', 'inca', 'english', 'pm', 'ci', 'system architecture', 'architect', 'stk calculations', 'coordination', 'gtest', 'cybersecurity', 'sell-side', 's4hana', 'intellij', 'ul4600', 'cable harness designer', 'finance', 'agile', 'nand', 'communication', 'canalyser', 'aspice', 'sw architect', 'ad platform', 'technology', 'engineering physics', 'invoice', 'test methods', 'safety', 'construction', 'customer requirement', 'mälardalen', 'software requirements', 'robot', "bachelor's degree", 'nfc', 'ble', 'integrity', 'rest', 'risk assessment', 'big data', 'c++', 'supply strategy', 'project leader', 'change management', 'design alternatives', 'spring', 'scrum'}

In [25]:
sw = stopwords.words('english')
sw[:3]

['i', 'me', 'my']

In [5]:
lemmatizer = WordNetLemmatizer()

- Tokenize
- Lemmatize
- Stopword
- Lowercase
- Remove outliers (?)
- Replace multispace

In [16]:
full_files = [f"data/resumes/{x}" for x in os.listdir('data/resumes')]
full_files_data = [open(x).read() for x in full_files]
full_files_assignment = [f"data/assignment/{x}/assignment.txt" for x in os.listdir('data/assignment')]
full_files_data_assignment = [open(x).read() for x in full_files_assignment if not 'annot' in x and not '.DS' in x]

In [17]:
multi_space = re.compile("\s+")

In [20]:
full_files_data = [multi_space.sub(' ', x) for x in full_files_data]
full_files_data_assignment = [multi_space.sub(' ', x) for x in full_files_data_assignment]

    Plotta CV i rymd (stem, lemma, extract keywords, embed)
    Plotta assignment i samma rymd (stem, lemma, extract keywords embed)
    Bestäm X (antal matchningar som önskas)
    Räkna ut hur många kluster som behövs (antal CV delat med X)
    Klustra
    Välj kluster som innehåller assignment

2:14
Ev använd dimensionality reduction (t.ex. PCA)

In [132]:
def clean(token):
    x = lemmatizer.lemmatize(token)
    x = '' if x in sw else x
    x = x.lower()
    return x

In [157]:
def clean_data(data):
    clean_x = [[clean(y) for y in x.split(' ')] for x in data]
    clean_x = [[y for y in x if len(y)] for x in clean_x]
    clean_x = [' '.join(x) for x in clean_x]
    return clean_x

In [158]:
clean_resume = clean_data(full_files_data)
clean_assignment = clean_data(full_files_data_assignment)

In [159]:
vectorizer = CountVectorizer(vocabulary=keywords)
vec_data = vectorizer.fit_transform(clean_resume+clean_assignment)

In [160]:
vec_resume = vec_data[:len(clean_resume)]
vec_assignment = vec_data[1037:]

In [168]:
svd = TruncatedSVD(n_components=100)

In [174]:
vec_svd = svd.fit_transform(vec_data)

In [186]:
clustering = KMeans(n_clusters=200).fit(vec_svd)

In [214]:
import numpy as np

In [218]:
np.where(clustering.predict(vec_svd) == 160)

(array([1037, 1038, 1041, 1066, 1075, 1091]),)

In [212]:
full_files_data_assignment[19]

'Project Manager, Reference 23105 Apply here Assignment Description and Requirements Project Manager_Treasury_Global Liquidity Management Tool The mission: It is your mission to run projects from idea to implementation. You are generally interested in technology, Business and IT deliverables and have a high interest in accelerating the digital agenda. Being responsive and enjoy working in a truly dynamic and international environment with global coordination describes you. Here we are developing and implementing a new process and solution for Treasury called Global Liquidity Management Tool for cash visibility, predictability and accessibility along with payment features and bank connections. We will retrieve data from different ERP systems and other external systems to achieve the required results. We have completed the RFP and are in the middle of the blueprint. We need your help to manage the project through the next phase starting with a pilot rollout before the full rollout. You a

In [124]:
# One Hot
vectorizer = OneHotEncoder()
vec_data = vectorizer.fit_transform(clean_resume+clean_assignment)

ValueError: Expected 2D array, got 1D array instead:
array=['financial control finance quality business financial technical management financial lead construction financial review financial manage'
 'financial fund management financial documentation management quality financial financial coordinating management quality financial planning financial planning ad implementation review review financial management control financial documentation management quality financial management management communication review quality network'
 'fund review financial management manufacturing financial financial technology sell-side financial management'
 ...
 'design safe design technical customer cross-functional collaborative quality driving architecture design safety control'
 'software developer planning planning coordinating software communication planning lead'
 'traction battery software software traction battery management battery battery battery safe diagnostics battery battery communication s